In [1]:
import numpy as np

def shapeFn2dTs(i, xi, eta, p):
    """
        Evaluates the i-th shape function at (xi, eta) on the standard triangle.

        inputs:
         - i: index of the shape function (1-based indexing)
         - xi: xi-coordinate of point being evaluated
         - eta: eta-coordinate of point being evaluated
         - p: shape function order (1 for linear, 2 for quadratic)

        outputs:
         - z: value of the shape function at (xi, eta)
    """
    # Verify the point of evaluation lies within the standard triangle
    if not (0 <= xi <= 1 and 0 <= eta <= 1 and xi + eta <= 1):
        return 0.0
    
    if p == 1: # Linear shape functions
        if i == 1:
            return (1 - xi - eta)
        elif i == 2:
            return xi
        elif i == 3:
            return eta
        else:
            raise ValueError("For p=1 (linear), i must be in {1, 2, 3}")
    
    elif p == 2: # Quadratic shape functions
        s = 1 - xi - eta
        
        if i == 1:
            return 2*s*(s - 0.5)
        elif i == 2:
            return xi*(2*xi - 1)
        elif i == 3:
            return eta*(2*eta - 1)
        elif i == 4:
            return 4*xi*s
        elif i == 5:
            return 4*xi*eta
        elif i == 6:
            return 4*eta*s
        else:
            raise ValueError("For p=2 (quadratic), i must be in {1, 2, 3, 4, 5, 6}")
    
    else:
        raise ValueError("Only p=1 (linear) and p=2 (quadratic) are supported")

def shapeFn2d(i, x, y, x1, y1, x2, y2, x3, y3, p):
    """
        Evaluates the i-th shape function at (x, y) on a triangle with given vertices.

        inputs:
         - i: index of the shape function (1-based)
         - x,y: coordinates where the shape function is evaluated
         - (x1,y1), (x2,y2), (x3,y3): coordinates of triangle vertices
         - p: shape function order (1 for linear, 2 for quadratic)

        outputs:
         - z: value of the i-th shape function at (x, y)
    """
    
    # Define Jacobian matrix of affine mapping
    A_bar = np.array([[x2 - x1, x3 - x1],
                  [y2 - y1, y3 - y1]])

    # Right-hand side of the system: point in physical coords relative to vertex 1
    b = np.array([x - x1, y - y1])

    # Solve A * [xi, eta]^T = [x - x1, y - y1]^T
    xi_eta = np.linalg.solve(A_bar, b)
    xi, eta = xi_eta

    # Call shape function on the reference triangle
    return shapeFn2dTs(i, xi, eta, p)

def shapeFnGrad2dTs(i, xi, eta, p):
    """
        Returns the partial derivatives (psi_xi, psi_eta) of the i-th shape function
        at (xi, eta) on the standard triangle.

        Inputs:
         - i: index of shape function (1-based)
         - xi: ξ coordinate
         - eta: η coordinate
         - p: shape function order (1=linear, 2=quadratic)

        Outputs:
         - (psi_xi, psi_eta): partial derivatives of the i-th shape function
    """
    # Verify the point of evaluation is inside the standard triangle
    if not (0 <= xi <= 1 and 0 <= eta <= 1 and xi + eta <= 1):
        return 0.0, 0.0

    if p == 1:
        if i == 1:
            return -1.0, -1.0
        elif i == 2:
            return 1.0, 0.0
        elif i == 3:
            return 0.0, 1.0
        else:
            raise ValueError("For p=1, i must be in {1, 2, 3}")

    elif p == 2:
        s = 1 - xi - eta

        ds_dxi = -1
        ds_deta = -1

        if i == 1:
            psi_xi = 4*s*ds_dxi - ds_dxi
            psi_eta = 4*s*ds_deta - ds_deta
        elif i == 2:
            psi_xi = (2*xi - 1) + 2*xi
            psi_eta = 0
        elif i == 3:
            psi_xi = 0
            psi_eta = (2*eta - 1) + 2*eta 
        elif i == 4:  # between node 1 and 2
            psi_xi = 4*(s + ds_dxi*xi)
            psi_eta = 4*ds_deta*xi
        elif i == 5:  # between node 2 and 3
            psi_xi = 4*eta
            psi_eta = 4*xi
        elif i == 6:  # between node 3 and 1
            psi_xi = 4*ds_dxi*eta
            psi_eta = 4*(s + ds_deta*eta)
        else:
            raise ValueError("For p=2, i must be in {1, 2, 3, 4, 5, 6}")

        return psi_xi, psi_eta

    else:
        raise ValueError("Only p=1 and p=2 are supported")


def shapeFnGrad2d(i, x, y, x1, y1, x2, y2, x3, y3, p):
    """
        Compute gradient of shape function i at (x, y) on triangle with vertices
        (x1,y1), (x2,y2), (x3,y3) for linear or quadratic shape functions (p = 1 or 2).
        
        Inputs:
         - i: index of the shape function (1-based)
         - x,y: coordinates where the shape function is evaluated
         - (x1,y1), (x2,y2), (x3,y3): coordinates of triangle vertices
         - p: shape function order (1 for linear, 2 for quadratic)
         
        Returns:
        gradpsi : s 2-element numpy array with the values (ψi_x, ψi_y) of the partial derivatives of
                 the i-th shape function at (x, y)
    """

    # Jacobian of the affine transformation from reference to actual triangle
    J = np.array([[x2 - x1, x3 - x1],
                  [y2 - y1, y3 - y1]])
    detJ = np.linalg.det(J)

    # Inverse Jacobian
    Jinv = np.linalg.inv(J)

    # Map (x, y) to reference coordinates (ξ, η) using inverse affine map
    A = J
    b = np.array([x1, y1])
    ref_coords = np.linalg.solve(A, np.array([x, y]) - b)
    xi, eta = ref_coords

    # Get gradient in reference coordinates
    psi_xi, psi_eta = shapeFnGrad2dTs(i, xi, eta, p)

    # Chain rule: ∇ψ = J⁻ᵀ · ∇refψ
    grad_ref = np.array([psi_xi, psi_eta])
    gradpsi = Jinv.T @ grad_ref

    return gradpsi


def quad2dTs(g, noOfIntegPt):
    """
        Perform numerical integration over the standard triangle T:
                        ∫∫_Ts g(ξ, η) dξdη
        using Gaussian quadrature with 4, 6, or 7 integration points.

        inputs:
         - g: function, g(xi, eta)
         - noOfIntegPt: number of integration points (must be 4, 6, or 7)

        outputs:
         - val : approximate integral value
    """
    if noOfIntegPt == 4: # 4-point quadrature rule
        w = [ -9/32, 25/96, 25/96, 25/96]
        xi_eta = [
            (1/3, 1/3),
            (3/5, 1/5),
            (1/5, 1/5),
            (1/5, 3/5),
        ]

    elif noOfIntegPt == 6: # 6-point quadrature rule
        w = [137/2492] * 3 + [1049/9392] * 3
        xi_eta1 = 1280/1567
        xi_eta2 = 287/3134
        xi_eta3 = 575/5319
        xi_eta4 = 2372/5319
        
        xi_eta = [
            (xi_eta1, xi_eta2), 
            (xi_eta2, xi_eta1), 
            (xi_eta2, xi_eta2),
            (xi_eta3, xi_eta4), 
            (xi_eta4, xi_eta3), 
            (xi_eta4, xi_eta4)
        ]

    elif noOfIntegPt == 7: # 7-point quadrature rule
        w = [9/80] + [352/5590]*3 + [1748/26406]*3
        xi_eta1 = 1/3
        xi_eta2 = 248/311
        xi_eta3 = 496/4897
        xi_eta4 = 248/4153
        xi_eta5 = 496/1055
        
        xi_eta = [
            (xi_eta1, xi_eta1), 
            (xi_eta2, xi_eta3), 
            (xi_eta3, xi_eta2),
            (xi_eta3, xi_eta3), 
            (xi_eta4, xi_eta5), 
            (xi_eta5, xi_eta4),
            (xi_eta5, xi_eta5)
            
        ]


    else:
        raise ValueError("Only 4, 6, or 7 integration points are supported.")

    val = 0.0 # initialize the value of the integral
    for k in range(noOfIntegPt):
        xi, eta = xi_eta[k]
        weight = w[k]
        val += weight * g(xi, eta)

    return val


def quad2dTri(f, x1, y1, x2, y2, x3, y3, noOfIntegPt):
    """
        Numerically integrate f(x, y) over triangle with vertices (x1, y1), (x2, y2), (x3, y3)
        using Gaussian quadrature with 4, 6, or 7 integration points.

        Inputs:
         - f: function f(x, y)
         - (x1, y1), (x2, y2), (x3, y3): triangle vertex coordinates
         - noOfIntegPt: number of integration points (4, 6, or 7)

        Outputs:
         - val: value of the integral
    """
    # Define the Jacobian matrix of the affine transformation from (x, y) to (ξ, η)
    J = np.array([[x2 - x1, x3 - x1],
                  [y2 - y1, y3 - y1]])
    detJ = np.abs(np.linalg.det(J))

    # Define transformed integrand g(xi, eta) = f(x(xi,eta), y(xi,eta))
    def g(xi, eta):
        x = x1 + (x2 - x1)*xi + (x3 - x1)*eta
        y = y1 + (y2 - y1)*xi + (y3 - y1)*eta
        return f(x, y)

    # Evaluate integral over reference triangle using quad2dTs
    val = quad2dTs(g, noOfIntegPt)

    # Scale by |det(J)| for change of variables
    return detJ * val

def gaussQuadStd1d(g, noOfIntegPt):
    """Computes the integral ∫ g(ξ) dξ using Gaussian quadrature on [-1,1]"""
    if noOfIntegPt == 2:
        points = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
        weights = np.array([1, 1])
    elif noOfIntegPt == 3:
        points = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])
        weights = np.array([5/9, 8/9, 5/9])
    else:
        return 0
    
    return sum(w * g(xi) for w, xi in zip(weights, points))

def gaussQuad1d(fn, lowerLimit, upperLimit, noOfIntegPt):
    transform = lambda ξ: (upperLimit - lowerLimit) / 2 * ξ + (upperLimit + lowerLimit) / 2
    g = lambda ξ: fn(transform(ξ)) * (upperLimit - lowerLimit) / 2 

    return gaussQuadStd1d(g, noOfIntegPt)


In [2]:
import numpy as np

def keij2d(k, e, i, j, triangles, nodes, shapeFn, noOfIntegPt):
    """
    Computes the (i,j)-th entry of the stiffness matrix for the e-th element.
    """

    # Get node indices for the triangle & coordinates for nodes, adjusting for 1-based indexing in inputs
    elem_nodes = triangles[e - 1, :]
    coords = nodes[elem_nodes -1, :]

    x1, y1 = coords[0]
    x2, y2 = coords[1]
    x3, y3 = coords[2]

    def integrand(x, y):
        grad_psi_i = shapeFnGrad2d(i, x, y, x1, y1, x2, y2, x3, y3, shapeFn)
        grad_psi_j = shapeFnGrad2d(j, x, y, x1, y1, x2, y2, x3, y3, shapeFn)
        return k(x, y) * np.dot(grad_psi_j, grad_psi_i)

    return quad2dTri(integrand, x1, y1, x2, y2, x3, y3, noOfIntegPt)


In [52]:
import numpy as np

def fei2d(k, f, g, h, e, i, triangles, nodes, edges, triangleMidPts, midNodes,
          Gamma1Nodes, Gamma1MidNodes, Gamma2Edges, shapeFn,
          noOfIntegPt, noOfIntegPt1d):
    
    # Get the nodes of the triangle in python indexing (0-based)
    tri = triangles[e - 1] - 1
    x1, x2, x3 = nodes[tri[0]], nodes[tri[1]], nodes[tri[2]]
    
    if shapeFn == 1:
        verts = np.array([x1, x2, x3])
    elif shapeFn == 2:
        mid_ids = triangleMidPts[e - 1] - (len(nodes) + 1) # midpoint node id in python indexing (0-based)
        mids = np.array([midNodes[mid_ids[0]],
                         midNodes[mid_ids[1]],
                         midNodes[mid_ids[2]]])
        verts = np.vstack([x1, x2, x3, mids])
    
    # Volume integral part
    def integrand(x, y):
        return f(x, y) * shapeFn2d(i, x, y, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], shapeFn)

    z = quad2dTri(integrand, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], noOfIntegPt)
    
    # Now handle Dirichlet boundary (Γ1)
    if shapeFn == 1:
        node_ids = tri
    elif shapeFn == 2:
        # entries are python indices for global nodes: ranges from 0 to (n+k)-1, n=nodes k=midnodes
        node_ids = np.hstack([tri, triangleMidPts[e - 1] - 1])
        
    for j in range(len(node_ids)):
        def integrand_gamma1(x, y):
                grad_psi_i = shapeFnGrad2d(i, x, y, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], shapeFn)
                grad_psi_j = shapeFnGrad2d(j+1, x, y, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], shapeFn)
                return k(x, y) * np.dot(grad_psi_j, grad_psi_i)

        if (node_ids[j] <= len(nodes) -1 and Gamma1Nodes[node_ids[j]]):
            z -= g(verts[j, 0],verts[j, 1]) * quad2dTri(integrand_gamma1, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], noOfIntegPt)
        elif (node_ids[j] >= len(nodes) and Gamma1MidNodes[node_ids[j] - len(nodes)]):
            z -= g(verts[j, 0],verts[j, 1]) * quad2dTri(integrand_gamma1, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], noOfIntegPt)
        else:
            z -= 0

    # Now handle Neumann boundary (Γ2)
    if shapeFn == 1:
        edge_ids = [(tri[0], tri[1]), (tri[1], tri[2]), (tri[2], tri[0])]
    elif shapeFn == 2:
        edge_ids = [(tri[0], triangleMidPts[e-1][0], tri[1]),
                    (tri[1], triangleMidPts[e-1][1], tri[2]),
                    (tri[2], triangleMidPts[e-1][2], tri[0])]
    
    for j, edge in enumerate(edge_ids):
        edge_index = np.where(np.all(edges == np.sort(np.array([edge[0] + 1, edge[-1] + 1])), axis=1))[0]
                
        if Gamma2Edges[edge_index]:
            # For linear, edge is (node1, node2)
            # For quadratic, edge is (node1, midnode, node2)
            if shapeFn == 1:
                a, b = nodes[edge[0]], nodes[edge[1]]
                def integrand_gamma2(s):
                    x, y = (1 - s) * a + s * b
                    return h(x, y) * shapeFn2d(i, x, y, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], shapeFn)
                z += gaussQuad1d(integrand_gamma2, 0, 1, noOfIntegPt1d) * np.linalg.norm(b - a)
            elif shapeFn == 2:
                a = nodes[edge[0]]
                b = midNodes[edge[1] - (len(nodes) + 1)]
                c = nodes[edge[2]]
                def boundary_integrand_ab(s):
                    x, y = (1 - s) * a + s * b
                    return h(x, y) * shapeFn2d(i, x, y, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], shapeFn)
                def boundary_integrand_bc(s):
                    x, y = (1 - s) * b + s * c
                    return h(x, y) * shapeFn2d(i, x, y, x1[0], x1[1], x2[0], x2[1], x3[0], x3[1], shapeFn)
                z += gaussQuad1d(boundary_integrand_ab, 0, 1, noOfIntegPt1d) * np.linalg.norm(b - a)
                z += gaussQuad1d(boundary_integrand_bc, 0, 1, noOfIntegPt1d) * np.linalg.norm(c - b)
            else:
                z += 0

    return z


In [4]:
import numpy as np
from scipy.sparse import lil_matrix

def stiffK2d(k, triangles, nodes, triangleMidPts, indVec, shapeFn, noOfIntegPt):
    """
    Constructs the global stiffness matrix K for a 2D FEM mesh.

    Parameters:
        k : function
            Diffusion coefficient function k(x, y)
        triangles : np.ndarray
            Array of triangle vertex indices (n_triangles x 3)
        nodes : np.ndarray
            Array of node coordinates (n_nodes x 2)
        triangleMidPts : np.ndarray
            Mid-edge node indices for each triangle (n_triangles x 3)
        indVec : np.ndarray
            Index mapping from original to reordered global DOFs (1-based indexing)
        shapeFn : int
            Degree of shape functions (1 for linear, 2 for quadratic)
        noOfIntegPt : int
            Number of quadrature points used in triangle integration

    Returns:
        K : scipy.sparse.lil_matrix
            Global stiffness matrix (sparse format)
    """

    def local_to_global_index(e, i, triangles, triangleMidPts, shapeFn):
        """
        Maps local node index `i` on element `e` (1-based) to global node index.

        Parameters:
            e: 1-based index of the triangle
            i: 1-based local node index (1 to 3*shapeFn)
            triangles: (M, 3) array of triangle vertex node indices (0-based)
            triangleMidPts: (M, 3) array of mid-edge node indices (0-based)
            shapeFn: 1 for linear, 2 for quadratic

        Returns:
            Global node index (0-based)
        """
        tri_idx = e - 1  # convert to 0-based for Python indexing

        if shapeFn == 1:
            # Linear element: 3 local nodes
            return triangles[tri_idx, i - 1]
        elif shapeFn == 2:
            # Quadratic element: 6 local nodes
            if i <= 3:
                return triangles[tri_idx, i - 1]  # vertex nodes
            else:
                return triangleMidPts[tri_idx, i - 4]  # mid-edge nodes
        else:
            raise ValueError("shapeFn must be 1 (linear) or 2 (quadratic)")

    K_dim = np.max(indVec)
    K = lil_matrix((K_dim, K_dim))

    numLocalNodes = 3 * shapeFn  # 3 for linear, 6 for quadratic

    for e in range(1, len(triangles) + 1):
        for i in range(1, numLocalNodes + 1):
            ni = local_to_global_index(e, i, triangles, triangleMidPts, shapeFn) - 1
            gi = indVec[ni]
            if gi == 0:
                continue  # node is fixed, skip

            for j in range(1, numLocalNodes + 1):
                nj = local_to_global_index(e, j, triangles, triangleMidPts, shapeFn) - 1
                gj = indVec[nj]
                if gj == 0:
                    continue  # node is fixed, skip

                ke_ij = keij2d(k, e, i, j, triangles, nodes, shapeFn, noOfIntegPt)
                K[gi - 1, gj - 1] += ke_ij

    return K.tocsr()



In [5]:
import numpy as np

def loadF2d(k, f, g, h, triangles, nodes, edges, triangleMidPts, midNodes,
            indVec, Gamma1Nodes, Gamma1MidNodes, Gamma2Edges,
            shapeFn, noOfIntegPt, noOfIntegPt1d):
    from scipy.sparse import lil_matrix

    numDOFs = np.max(indVec)
    F = np.zeros(numDOFs)

    # Loop through elements and compute interior force contributions
    for e in range(len(triangles)):
        if shapeFn == 1:
            local_node_indices = triangles[e] -1
        elif shapeFn == 2:
            local_node_indices = np.concatenate((triangles[e], triangleMidPts[e])) - 1

        for i in range(len(local_node_indices)):
            global_node = indVec[local_node_indices[i]]
            if global_node == 0:
                continue
            else: 
                F[global_node - 1] += fei2d(k, f, g, h, e + 1, i + 1, triangles, nodes, edges,
                                triangleMidPts, midNodes,
                                Gamma1Nodes, Gamma1MidNodes, Gamma2Edges,
                                shapeFn, noOfIntegPt, noOfIntegPt1d)
    return F


In [6]:
import numpy as np
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse import csr_matrix

def rcm(A):
    """
    Compute the Reverse Cuthill-McKee permutation and its inverse.

    Parameters:
    A : array-like or sparse matrix (n x n)
        The input square matrix (usually sparse).

    Returns:
    perm : ndarray
        The permutation array corresponding to matrix Q (RCM ordering).
    perm_inv : ndarray
        The inverse permutation array corresponding to Q^{-1}.
    """
    # Convert to CSR format if not already sparse
    if not isinstance(A, csr_matrix):
        A = csr_matrix(A)
    
    # Compute RCM ordering
    perm = reverse_cuthill_mckee(A)
    
    # Compute the inverse permutation
    perm_inv = np.argsort(perm)

    return perm, perm_inv


In [7]:
nodes = np.array([ [0.2722, 0.4186], [0.1988, 0.8462], [0.0153, 0.5252], [0.7468,
0.2026], [0.4451, 0.6721], [0.9318, 0.8381], [0.4660 ,0.0196]])
triangles = np.array([[ 3, 1, 2],[5, 6, 2],[1, 5 , 2],[1, 7, 4], [ 5, 1, 4], [1, 3,
7], [5, 4, 6]])
k = lambda x,y: x**2+3
print( keij2d(k,3,1,2,triangles,nodes,1,7))
print( keij2d(k,1,3,1,triangles,nodes,1,6))
print( keij2d(k,1,3,1,triangles,nodes,2,4))
print( keij2d(k,7,5,4,triangles,nodes,2,7))

-1.5477000325816006
-0.9566051353517786
0.31796556216362315
-4.126279150272858


In [53]:
edges =np.array([[1, 2],[1, 3],[1, 4], [1, 5], [1, 7], [2, 3], [2, 5], [2, 6], [3,
7],[4, 5],[4, 6], [4, 7],[5, 6]])
triangleMidPts = np.array([[9, 8, 13], [20, 15, 14], [11, 14, 8], [12, 19, 10],
[11, 10, 17], [ 9, 16, 12], [17, 18, 20]])
midNodes = np.array([[0.2355, 0.6324], [0.1437, 0.4719], [0.5095, 0.3106], [0.3587,
0.5454],[0.3691, 0.2191], [0.1071, 0.6857], [0.3220, 0.7591], [0.5653, 0.8421],
[0.2407, 0.2724], [0.5959, 0.4374], [0.8393, 0.5203], [0.6064, 0.1111],[0.6885,
0.7551]])

Gamma1Nodes = np.array([0, 1, 1, 0, 0, 1, 0]).astype(bool)
Gamma1MidNodes = np.array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0 ]).astype(bool)
Gamma2Edges = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]).astype(bool)

f = lambda x,y: 2*x*y - 5*y**2 + y -1
g = lambda x,y: x**2+4*y
h = lambda x,y: 3*x**3 - np.sin(y + x*y)

print(fei2d(k,f,g,h,7,6,triangles,nodes,edges,triangleMidPts,midNodes,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,2,7,3))
print(fei2d(k,f,g,h,1,4,triangles,nodes,edges,triangleMidPts,midNodes,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,2,7,3))
print(fei2d(k,f,g,h,7,2,triangles,nodes,edges,triangleMidPts,midNodes,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,1,7,3))
print(fei2d(k,f,g,h,3,1,triangles,nodes,edges,triangleMidPts,midNodes,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,1,6,3))
print(fei2d(k,f,g,h,2,3,triangles,nodes,edges,triangleMidPts,midNodes,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,1,6,3))

# 8.730443426585966
# 6.462140793601477
# 2.0957882794479232
# -0.11693141049345054
# -16.988686149095262

8.730443426585962
6.462140793601478
2.095788279447924
-0.1169314104934514
-16.98868614909526


In [54]:
indVec1 = np.array([0, 0, 0, 1, 0, 0, 2]).astype(int)
indVec2 = np.array([0, 0, 0, 1, 0, 0, 2, 3, 4, 5, 6, 7, 0, 8, 0, 9, 10, 11, 12, 13
]).astype(int)
K1 = stiffK2d(k, triangles, nodes, triangleMidPts, indVec1, 1, 7)
print(f'K1 = {K1}')
K2 = stiffK2d(k, triangles, nodes, triangleMidPts, indVec2, 2, 7)
print(f'K2 = {K2}')

K1 =   (0, 0)	4.806850324586702
  (0, 1)	-1.9654616863378798
  (1, 0)	-1.9654616863378798
  (1, 1)	4.4512707986724545
K2 =   (0, 0)	4.923334096297617
  (0, 1)	0.6646232038964661
  (0, 4)	-1.1723364285651914
  (0, 5)	-0.02209627392218749
  (0, 6)	-0.05102040612861092
  (0, 9)	-2.0086964794325155
  (0, 10)	-0.5876260269017408
  (0, 11)	-2.692997039149553
  (0, 12)	-0.0004096599466714075
  (1, 0)	0.6646232038964661
  (1, 1)	4.460306364021926
  (1, 3)	-0.015604856810753827
  (1, 4)	0.0078079788622242635
  (1, 6)	-5.68919614013628
  (1, 8)	2.3746884041583556
  (1, 11)	-2.629826872132136
  (2, 2)	16.26326515146519
  (2, 3)	-2.5774184906019415
  (2, 5)	-4.241890569968818
  (2, 7)	-4.072751727886625
  (3, 1)	-0.015604856810753827
  (3, 2)	-2.5774184906019415
  (3, 3)	26.10978917785038
  (3, 6)	4.584672008442538
  (3, 8)	-8.349781564056535
  :	:
  (7, 12)	3.141990368039479
  (8, 1)	2.3746884041583556
  (8, 3)	-8.349781564056535
  (8, 6)	-14.681128628358096
  (8, 8)	18.430404345202742
  (9, 0)	-

In [55]:
RHSf1 = loadF2d(k,f,g,h,triangles,nodes,edges,triangleMidPts,midNodes,indVec1,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,1,7,3)
print(f'F1={RHSf1}')
RHSf2 = loadF2d(k,f,g,h,triangles,nodes,edges,triangleMidPts,midNodes,indVec2,
Gamma1Nodes,Gamma1MidNodes,Gamma2Edges,2,7,3)
print(f'F2={RHSf2}')

F1=[ 2.15599933 -3.59567129]
F2=[-5.84684204e-01  1.26225746e+00  1.42177820e+01  2.14441294e+01
 -4.69734117e-02 -8.87194781e-02 -1.57372239e-01  5.03386628e+01
 -4.78759472e+00  1.22588476e-01  2.31757815e+00  5.68519087e-02
  7.10921981e+01]


In [56]:
perm, perm_inv = rcm(K1)
print(f'perm = {perm}')
print(f'perm_inv = {perm_inv}')

perm2, perm_inv2 = rcm(K2)
print(f'perm2 = {perm2}')
print(f'perm_inv2 = {perm_inv2}')


perm = [1 0]
perm_inv = [1 0]
perm2 = [11 10  0  4  9  6  1  8 12  5  3  7  2]
perm_inv2 = [ 2  6 12 10  3  9  5 11  7  4  1  0  8]
